In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from sympy import Symbol, symbols, Eq, Function #, init_printing; init_printing()
R, Z, Phi = symbols("R, Z, \phi", real=True)
x0_R, x0_Z = symbols("x_{0R}, x_{0Z}", real=True)
X_R, X_Z = [Function(latexstr, real=True)(x0_R, x0_Z, Phi) for latexstr in ["X_R", "X_Z"] ]
X_Phi = Symbol("X_{\phi}", real=True)
XRdot = Function("\dot{X}_{R}", real=True)(X_R, X_Z, X_Phi)
# XRdot, XZdot = symbols("$X_R$, dot{X_Z}", cls=Function)

In [4]:
XRdot

\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi})

In [5]:
from sympy import Array
from sympy.core.numbers import Integer
from sympy.core.function import Derivative
from sympy.core.power import Pow
ndiff_R, ndiff_Z = 1, 1
result = XRdot.diff(x0_R, ndiff_R, x0_Z, ndiff_Z)

import numpy as np
import pandas as pd
ndiff = ndiff_R + ndiff_Z
XRZ_index = ['C']
for idiff in range(1, ndiff+1):
    for idiff_R in range(idiff+1):
        idiff_Z = idiff - idiff_R
        XRZ_index.append('XR['+str(idiff_R)+','+str(idiff_Z)+']')
        XRZ_index.append('XZ['+str(idiff_R)+','+str(idiff_Z)+']')


# for R_diff_order in range(ndiff_R+ndiff_Z, -1, -1):
#     for Z_diff_order in range(ndiff_R+ndiff_Z -R_diff_order, -1, -1 ):
#         if R_diff_order >= 2:
#             R_partdiff_substr = "\partial_{R}^{"+f"{R_diff_order}"+"}"
#         elif R_diff_order == 1:
#             R_partdiff_substr = "\partial_{R}"
#         elif R_diff_order == 0:
#             R_partdiff_substr = ""
            
#         if Z_diff_order >= 2:
#             Z_partdiff_substr = "\partial_{Z}^{"+f"{Z_diff_order}"+"}"
#         elif Z_diff_order == 1:
#             Z_partdiff_substr = "\partial_{Z}"
#         elif Z_diff_order == 0:
#             Z_partdiff_substr = ""
            
#         result = result.subs(
#             XRdot.diff(X_R, R_diff_order, X_Z, Z_diff_order), 
#             Symbol(R_partdiff_substr + Z_partdiff_substr + " \dot{X}_{R}", real=True))

# for x0_R_diff_order in range(ndiff_R, -1, -1):
#     for x0_Z_diff_order in range(ndiff_Z, -1, -1):
#         if x0_R_diff_order >= 2:
#             x0_R_partdiff_substr = "\partial_{x_{0R}}^{"+f"{x0_R_diff_order}"+"}"
#         elif x0_R_diff_order == 1:
#             x0_R_partdiff_substr = "\partial_{x_{0R}}"
#         elif x0_R_diff_order == 0:
#             x0_R_partdiff_substr = ""
            
#         if x0_Z_diff_order >= 2:
#             x0_Z_partdiff_substr = "\partial_{x_{0Z}}^{"+f"{x0_Z_diff_order}"+"}"
#         elif x0_Z_diff_order == 1:
#             x0_Z_partdiff_substr = "\partial_{x_{0Z}}"
#         elif x0_Z_diff_order == 0:
#             x0_Z_partdiff_substr = ""
            
#         result = result.subs(
#             X_R.diff(x0_R, x0_R_diff_order, x0_Z, x0_Z_diff_order),
#             Symbol(x0_R_partdiff_substr+x0_Z_partdiff_substr+" X_{R}",real=True) )
    
#         result = result.subs(
#             X_Z.diff(x0_R, x0_R_diff_order, x0_Z, x0_Z_diff_order),
#             Symbol(x0_R_partdiff_substr+x0_Z_partdiff_substr+" X_{Z}",real=True) )

print("We show them term-by-term as the following:")
factored_result = result.factor()
how_many_terms_in_result = len(factored_result.args)
XRZ_df = pd.DataFrame(np.zeros( (len(XRZ_index),how_many_terms_in_result ), dtype=int), index= XRZ_index)
for i, term in enumerate(factored_result.args):
    term
    for factor in term.args:
        if factor.func is Integer:
            combinatorial_coeff = factor # Combinatorial number
            XRZ_df.at['C', i] = combinatorial_coeff
            continue
        
        if factor.func is Pow:
            factor_inside_power = factor.args[0]
            factor_power = factor.args[1]
        else:
            factor_inside_power = factor
            factor_power = 1
        
        assert(factor_inside_power.func is Derivative)
        if factor_inside_power.args[0] == XRdot:
            continue
        
        if factor_inside_power.args[0] == X_R:
            XR_or_XZ = 'XR'
        elif factor_inside_power.args[0] == X_Z:
            XR_or_XZ = 'XZ'
        
        x0_R_diff_n = 0
        x0_Z_diff_n = 0
        for xdiff in factor_inside_power.args[1:]:
            if xdiff[0] == x0_R:
                x0_R_diff_n = xdiff[1]
            elif xdiff[0] == x0_Z:
                x0_Z_diff_n = xdiff[1]
        
        XRZ_df.at[XR_or_XZ+'['+str(x0_R_diff_n)+','+str(x0_Z_diff_n)+']', i] = factor_power
    if XRZ_df.at['C', i] == 0:
        XRZ_df.at['C', i] = 1

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
# df.head()

XRZ_df

We show them term-by-term as the following:


Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), X_R(x_{0R}, x_{0Z}, \phi))*Derivative(X_R(x_{0R}, x_{0Z}, \phi), x_{0R}, x_{0Z})

Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), X_Z(x_{0R}, x_{0Z}, \phi))*Derivative(X_Z(x_{0R}, x_{0Z}, \phi), x_{0R}, x_{0Z})

Derivative(X_R(x_{0R}, x_{0Z}, \phi), x_{0R})*Derivative(X_R(x_{0R}, x_{0Z}, \phi), x_{0Z})*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), (X_R(x_{0R}, x_{0Z}, \phi), 2))

Derivative(X_R(x_{0R}, x_{0Z}, \phi), x_{0R})*Derivative(X_Z(x_{0R}, x_{0Z}, \phi), x_{0Z})*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi))

Derivative(X_R(x_{0R}, x_{0Z}, \phi), x_{0Z})*Derivative(X_Z(x_{0R}, x_{0Z}, \phi), x_{0R})*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi))

Derivative(X_Z(x_{0R}, x_{0Z}, \phi), x_{0R})*Derivative(X_Z(x_{0R}, x_{0Z}, \phi), x_{0Z})*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), (X_Z(x_{0R}, x_{0Z}, \phi), 2))

,0,1,2,3,4,5
C,1,1,1,1,1,1
"XR[0,1]",0,0,1,0,1,0
"XZ[0,1]",0,0,0,1,0,1
"XR[1,0]",0,0,1,1,0,0
"XZ[1,0]",0,0,0,0,1,1
"XR[0,2]",0,0,0,0,0,0
"XZ[0,2]",0,0,0,0,0,0
"XR[1,1]",1,0,0,0,0,0
"XZ[1,1]",0,1,0,0,0,0
"XR[2,0]",0,0,0,0,0,0


In [5]:

# for X_R_x0_R_diff_order in range(ndiff_R, -1, -1):
#     for X_R_x0_Z_diff_order in range(ndiff_Z, -1, -1):
#         for X_Z_x0_R_diff_order in range(ndiff_R, -1, -1):
#             for X_Z_x0_Z_diff_order in range(ndiff_Z, -1, -1):
#                 for term in result.factor().args:

#                     if x0_R_diff_order >= 2:
#                         x0_R_partdiff_substr = "\partial_{x_{0R}}^{"+f"{x0_R_diff_order}"+"}"
#                     elif x0_R_diff_order == 1:
#                         x0_R_partdiff_substr = "\partial_{x_{0R}}"
#                     elif x0_R_diff_order == 0:
#                         x0_R_partdiff_substr = ""

#                     if x0_Z_diff_order >= 2:
#                         x0_Z_partdiff_substr = "\partial_{x_{0Z}}^{"+f"{x0_Z_diff_order}"+"}"
#                     elif x0_Z_diff_order == 1:
#                         x0_Z_partdiff_substr = "\partial_{x_{0Z}}"
#                     elif x0_Z_diff_order == 0:
#                         x0_Z_partdiff_substr = ""

#                     result = result.subs(
#                         X_R.diff(x0_R, x0_R_diff_order, x0_Z, x0_Z_diff_order),
#                         Symbol(x0_R_partdiff_substr+x0_Z_partdiff_substr+" X_{R}",real=True) )

#                     result = result.subs(
#                         X_Z.diff(x0_R, x0_R_diff_order, x0_Z, x0_Z_diff_order),
#                         Symbol(x0_R_partdiff_substr+x0_Z_partdiff_substr+" X_{Z}",real=True) )

In [5]:
XRdot.diff(X_R).subs(XRdot.diff(X_R), Symbol("\partial_{1} \dot{X}_R", real=True))

\partial_{1} \dot{X}_R

In [6]:
from sympy import FiniteSet
len(FiniteSet(FiniteSet(5,5,7),FiniteSet(3,6)))

2

In [7]:
from sympy import FiniteSet
def list_of_all_partitions(set_to_partition):
    
    def partition_into_two_parts(set_to_split):
        set_size = len(set_to_split)
        all_partitions = []
        list_elements = list(set_to_split)
        first_ele = list_elements[0]
        for i in range( 1, 2**(set_size-1) ): # we skip i==0, because that is full set
            first_part = [first_ele]
            second_part = [ ]
            for j, ele in enumerate(list_elements[1:]):
                if (i >> j) % 2 == 0:
                    first_part.append(ele)
                else:
                    second_part.append(ele)
            if len(second_part)!=0:
                all_partitions.append([FiniteSet(*first_part), FiniteSet(*second_part)])
    #             all_partitions.append(FiniteSet(
    #                 FiniteSet(*first_part), FiniteSet(*second_part)))
            else:
                all_partitions.append( [FiniteSet(*first_part)] )
    #             all_partitions.append(FiniteSet(
    #                 FiniteSet(*first_part) ))

        return all_partitions # FiniteSet(*all_partitions)
    all_partitions = [ [set_to_partition] ]
    binary_partitions = partition_into_two_parts(set_to_partition)
    for bi_part in binary_partitions:
        if len(bi_part[-1])>1:
            first_part = bi_part[0]
            for secondpart_partition in list_of_all_partitions(bi_part[-1]):
                all_partitions.append([first_part] + secondpart_partition)
        else:
            all_partitions.append(bi_part)
    return all_partitions


In [8]:
list(FiniteSet(FiniteSet(5),1,2))

[FiniteSet(5), 1, 2]

In [9]:
from random import randrange
from sympy import derive_by_array, Intersection, tensorproduct, tensorcontraction
ndiff_R, ndiff_Z = 3, 0
ndiff = ndiff_R+ndiff_Z
sum_over_big_pi = 0
for small_pi in list_of_all_partitions(  FiniteSet(*list(range(1, ndiff+1))) ):
    small_pi_size = len(small_pi)
    repeated_grad_result = XRdot
    for _ in range(small_pi_size):
        repeated_grad_result = derive_by_array(repeated_grad_result, [X_R, X_Z])
    
    term_of_small_pi = repeated_grad_result
    for iB, B in enumerate(small_pi):
        B_size = len(B)
        _ndiff_R = len(Intersection(  B, FiniteSet(*list(range(1, ndiff_R+1))) ))
        _ndiff_Z = len(Intersection(  B, FiniteSet(*list(range(ndiff_R+1, ndiff+1))) ))
        term_of_small_pi = tensorcontraction(tensorproduct(
            term_of_small_pi, 
            [X_R.diff(x0_R, _ndiff_R, x0_Z, _ndiff_Z), X_Z.diff(x0_R, _ndiff_R, x0_Z, _ndiff_Z)]), 
          (randrange(small_pi_size - iB), small_pi_size - iB) ) # (small_pi_size - iB -1, small_pi_size - iB) 
    sum_over_big_pi += term_of_small_pi
result = sum_over_big_pi

In [10]:
print("We show them term-by-term as the following:")
for term in result.factor().args:
    term

We show them term-by-term as the following:


Derivative(X_R(x_{0R}, x_{0Z}, \phi), x_{0R})**3*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), (X_R(x_{0R}, x_{0Z}, \phi), 3))

Derivative(X_Z(x_{0R}, x_{0Z}, \phi), x_{0R})**3*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), (X_Z(x_{0R}, x_{0Z}, \phi), 3))

Derivative(X_R(x_{0R}, x_{0Z}, \phi), (x_{0R}, 3))*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), X_R(x_{0R}, x_{0Z}, \phi))

Derivative(X_Z(x_{0R}, x_{0Z}, \phi), (x_{0R}, 3))*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), X_Z(x_{0R}, x_{0Z}, \phi))

...

3*Derivative(X_R(x_{0R}, x_{0Z}, \phi), x_{0R})*Derivative(X_Z(x_{0R}, x_{0Z}, \phi), x_{0R})**2*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), X_R(x_{0R}, x_{0Z}, \phi), (X_Z(x_{0R}, x_{0Z}, \phi), 2))

3*Derivative(X_R(x_{0R}, x_{0Z}, \phi), x_{0R})*Derivative(X_R(x_{0R}, x_{0Z}, \phi), (x_{0R}, 2))*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), (X_R(x_{0R}, x_{0Z}, \phi), 2))

3*Derivative(X_R(x_{0R}, x_{0Z}, \phi), x_{0R})*Derivative(X_Z(x_{0R}, x_{0Z}, \phi), (x_{0R}, 2))*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi))

3*Derivative(X_R(x_{0R}, x_{0Z}, \phi), (x_{0R}, 2))*Derivative(X_Z(x_{0R}, x_{0Z}, \phi), x_{0R})*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi))

3*Derivative(X_Z(x_{0R}, x_{0Z}, \phi), x_{0R})*Derivative(X_Z(x_{0R}, x_{0Z}, \phi), (x_{0R}, 2))*Derivative(\dot{X}_{R}(X_R(x_{0R}, x_{0Z}, \phi), X_Z(x_{0R}, x_{0Z}, \phi), X_{\phi}), (X_Z(x_{0R}, x_{0Z}, \phi), 2))

In [92]:
list_of_all_partitions(  FiniteSet(*list(range(1, ndiff+1))) )

[[FiniteSet(1, 2, 3)],
 [FiniteSet(1, 3), FiniteSet(2)],
 [FiniteSet(1, 2), FiniteSet(3)],
 [FiniteSet(1), FiniteSet(2, 3)],
 [FiniteSet(1), FiniteSet(2, 3)],
 [FiniteSet(1), FiniteSet(2), FiniteSet(3)]]